# Notebook 2: Conducting and Evaluating Regression Analysis

In [49]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import os
import tqdm
import glob
import pandas as pd
import sklearn

np.random.seed(1)

Import datasets that were preprocessed in Notebook 1

In [50]:
wb_data = pd.read_csv("data/wb_data.csv")
wb_data_short = pd.read_csv("data/wb_data_short.csv")
whr_data = pd.read_csv("data/whr_data.csv")

wb_data.index = wb_data["Country Name"]
wb_data.drop(columns=["Country Name", "Country Name.1"], inplace=True)
wb_data_short.index = wb_data_short["Country Name"]
wb_data_short.drop(columns=["Country Name", "Country Name.1"], inplace=True)


whr_data.index = whr_data["Country name"]
whr_data.drop(columns=["Country name", "Country name.1"], inplace=True)
#whr_data.head(20)

# sort by index
wb_data.sort_index(inplace=True)
wb_data_short.sort_index(inplace=True)
whr_data.sort_index(inplace=True)

In [51]:
# test: are the datasets equal
print(sorted(list(wb_data.index))==sorted(list(whr_data.index)))

True


In [ ]:
# drop everything but life satisfaction ladder score from whr data
whr_scores = whr_data["Ladder score"]

## Pearson correlation coefficients
Aim: Get the correlation coefficient of each indicator with the whr_data

In [62]:
import scipy.stats

indicator_corr = wb_data_short.corr(method="pearson")
indicator_corr
indicator_corr[indicator_corr>0.8]

indicator_corr_list = []

#print(sorted(indicator_corr[indicator_corr>0.8].tolist()))

AttributeError: 'DataFrame' object has no attribute 'tolist'

## Split data into train and test set

In [ ]:
test_size = 30

def split_data(data, gt, test_size):
    """
    split dataset into train and test set
    
    returns: tuple of numpy arrays (train_set, test_set)
    """
    test_set = data.sample(n=test_size)
    test_country_names = list(test_set.index.values)
    train_set = data.drop(labels=test_country_names)
    
    test_gt = gt.loc[test_set.index.values]
    train_gt = gt.drop(labels=test_country_names)
    
    return train_set, test_set, train_gt, test_gt

train, test, train_gt, test_gt = split_data(wb_data_short, whr_scores, test_size)

In [ ]:
print(train.shape, test.shape, train_gt.shape, test_gt.shape)
print(list(test.index)==list(test_gt.index))

## Linear regression

In [ ]:
from sklearn.linear_model import LinearRegression

def n_fold_ceval(n, data, gt, test_size, loss):
    """
    perform n-fold validation
    
    args: number of validations (n), dataset of indicators (data), groundtruth data (gt), size of test set (test size), loss function (loss)
    returns: list of length n, each entry contains loss for one validation loop
    """
    loss_list = []
    for i in range(0,n):
        train, test, train_gt, test_gt = split_data(data, gt, test_size)
        
        reg = LinearRegression().fit(train, train_gt)
        test_pred = reg.predict(test)
        loss = sklearn.metrics.mean_squared_error(test_gt, test_pred)
        loss_list.append(loss)
        
    return loss_list

Let's see how linear regression performs on wb_data and wb_data_short (redundant indicators removed).

In [ ]:
losses = np.array(n_fold_ceval(1500, wb_data, whr_scores, 30))
losses_mean = losses.mean()

print(losses_mean)
print(losses[0:50])

For high n, the mse-loss is around 5. We print the first 50 entries of the loss array to check for outliers. It turns out that the variance is quite large and the loss is roughly in a range of [1, 25]